In [510]:
import pandas as pd
import numpy as np

In [511]:
cust_data=pd.read_csv('Customer_Data_re.csv',low_memory=False)
inv_data=pd.read_csv('Final_invoice.csv',low_memory=False)
car_val=pd.read_csv('scrap car price.csv',index_col=['car'])
jtd_data=pd.read_csv('JTD.csv')

In [512]:
cust_data.drop(['Title', 'Marital Status', 'Occupation', 'Date of Birth', 'Death date'],axis=1,inplace=True)

In [513]:
cust_data['Partner Type'].fillna(value=float(cust_data['Partner Type'].mode().values),axis=0,inplace=True)

In [514]:
cust_data['Data Origin'].fillna(method='ffill',axis=0,inplace=True,limit =60)
cust_data['Data Origin'].fillna(method='bfill',axis=0,inplace=True)


In [515]:
inv_data['min_price']=inv_data['Model'].map(car_val['min_price'])
inv_data['max_price']=inv_data['Model'].map(car_val['max_price'])

In [516]:
inv_data.loc[inv_data['max_price']<500000,'car_cat']='low range'
inv_data.loc[(inv_data['max_price']>500000 )&(inv_data['max_price']<1500000),'car_cat']='mid range'
inv_data.loc[inv_data['max_price']>1500000,'car_cat']='high range'

In [517]:
max_mid_range_mean=inv_data[inv_data['car_cat']=='mid range']['max_price'].mean()
min_mid_range_mean=inv_data[inv_data['car_cat']=='mid range']['min_price'].mean()

In [518]:
inv_data['max_price'].fillna(value=max_mid_range_mean,axis=0,inplace=True)
inv_data['min_price'].fillna(value=min_mid_range_mean,axis=0,inplace=True)
inv_data['car_cat'].fillna(value='mid range',axis=0,inplace=True)

In [519]:
inv_data.drop(inv_data.columns[np.where(inv_data.isnull().sum()>15000)],axis=1,inplace=True)

In [520]:
inv_data['Model'].fillna(method='ffill',axis=0,inplace=True,limit =700)
inv_data['Model'].fillna(method='bfill',axis=0,inplace=True)


In [521]:
inv_data.dropna(axis=0,inplace=True)

In [522]:
jtd_data.drop(['Labor Value Number'],axis=1,inplace=True)

In [523]:
jtd_data.dropna(axis=0,inplace=True)

In [524]:
cust_inc_data=inv_data.merge(cust_data,on='Customer No.',how='left')

In [525]:
cust_inc_data.dropna(axis=0,inplace=True)

In [526]:
final_comb_data=cust_inc_data.merge(jtd_data,left_on='Job Card No',right_on='DBM Order',how='left')

In [527]:
final_comb_data.dropna(axis=0,inplace=True)

In [528]:
final_comb_data['Invoice Date']=pd.to_datetime(final_comb_data['Invoice Date'])

In [529]:
final_comb_data['Invoice Date Time']=final_comb_data['Invoice Date'].astype(str).str.cat(final_comb_data[['Invoice Time']].astype(str),sep=" ")

In [530]:
final_comb_data['JobCard Date']=pd.to_datetime(final_comb_data['JobCard Date'])

In [532]:
final_comb_data['JobCard Date Time']=final_comb_data['JobCard Date'].astype(str).str.cat(final_comb_data[['JobCard Time']].astype(str),sep=" ")

In [533]:
final_comb_data['JobCard Date Time']=pd.to_datetime(final_comb_data['JobCard Date Time'])
final_comb_data['Invoice Date Time']=pd.to_datetime(final_comb_data['Invoice Date Time'])

In [535]:
final_comb_data['Service Time']=final_comb_data['Invoice Date Time']-final_comb_data['JobCard Date Time']

In [537]:

final_comb_data.loc[final_comb_data['Service Time']<pd.Timedelta(hours=12),'Service Category']='Super_fast'
final_comb_data.loc[(final_comb_data['Service Time']>pd.Timedelta(hours=12)) & (final_comb_data['Service Time']<pd.Timedelta(hours=36)),'Service Category']='fast'
final_comb_data.loc[(final_comb_data['Service Time']>pd.Timedelta(hours=36))& (final_comb_data['Service Time']<pd.Timedelta(hours=100)),'Service Category']='moderate'
final_comb_data.loc[(final_comb_data['Service Time']>pd.Timedelta(hours=100))& (final_comb_data['Service Time']<pd.Timedelta(hours=192)),'Service Category']='slow'
final_comb_data.loc[(final_comb_data['Service Time']>pd.Timedelta(hours=192))& (final_comb_data['Service Time']<pd.Timedelta(hours=360)),'Service Category']='Super_slow'
final_comb_data.loc[final_comb_data['Service Time']>pd.Timedelta(hours=360),'Service Category']='Default'

In [549]:
final_comb_data.drop(['Customer No.','Gate Pass Time','Invoice Date', 'Invoice No', 'Invoice Time','Job Card No', 'JobCard Date', 'JobCard Time','Pin code','Recovrbl Exp', 'Regn No','User ID','Unnamed: 0_x','Partner Type','Unnamed: 0_y','DBM Order','Material','Target quantity UoM','Invoice Date Time', 'JobCard Date Time', 'Service Time','Make','Plant Name1','Print Status''Business Partner''District'],axis=1,inplace=True)

In [550]:
final_comb_data.to_csv('final.csv',index=False)